# Mnist

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'Mnist'

df = pd.read_csv(f'LLaVa_{dataset}_results.csv')

In [4]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [5]:
illusionless_indexes

[66,
 878,
 988,
 30,
 422,
 947,
 1006,
 568,
 328,
 70,
 1066,
 1003,
 671,
 155,
 511,
 739,
 91,
 860,
 284,
 727,
 781,
 862,
 580,
 537,
 935,
 357,
 620,
 742,
 272,
 490,
 900,
 768,
 90,
 8,
 482,
 274,
 399,
 1098,
 749,
 491,
 643,
 922,
 892,
 962,
 133,
 664,
 1027,
 320,
 459,
 845,
 488,
 75,
 65,
 1017,
 616,
 147,
 1092,
 165,
 306,
 787,
 766,
 231,
 197,
 904,
 340,
 391,
 714,
 887,
 848,
 912,
 502,
 562,
 293,
 1068,
 365,
 244,
 546,
 933,
 619,
 337,
 356,
 360,
 983,
 711,
 890,
 458,
 10,
 89,
 679,
 654,
 496,
 163,
 536,
 917,
 829,
 324,
 799,
 1015,
 494,
 1075,
 556,
 1065,
 1023,
 128,
 1002,
 945,
 821,
 278,
 861,
 1108]

In [6]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,LLaVa_Mnist_ill_images,LLaVa_Mnist_illusion_images_filtered,LLaVa_Mnist_illusionless_images,LLaVa_Mnist_illusionless_images_filtered,LLaVa_Mnist_raw_images
0,Mnist_1,A field of blooming sunflowers swaying in the ...,low quality,1.5,7,No illusion,No illusion,No illusion,No illusion,'digit 7'
1,Mnist_2,A peaceful countryside scene with grazing shee...,low quality,1.5,2,No illusion,No illusion,No illusion,No illusion,'digit 2'
2,Mnist_3,A tranquil pond with lily pads floating on the...,low quality,1.5,1,No illusion,No illusion,No illusion,No illusion,'digit 0'
3,Mnist_4,A sunny vineyard with rows of ripe grapes,low quality,1.5,0,No illusion,No illusion,No illusion,No illusion,'digit 0'
4,Mnist_5,A picturesque vineyard at sunset with the sky ...,low quality,1.5,4,No illusion,No illusion,No illusion,No illusion,'digit 4'
...,...,...,...,...,...,...,...,...,...,...
1104,Mnist_1105,Misty jungle surrounded by vibrant flowers and...,low quality,1.5,8,No illusion,No illusion,No illusion,No illusion,'digit 0'
1105,Mnist_1106,A forest with blooming flowers,low quality,1.5,0,No illusion,No illusion,No illusion,No illusion,'digit 0'
1106,Mnist_1107,"Desolate desert landscape, shifting sands illu...",low quality,1.5,5,No illusion,No illusion,No illusion,No illusion,'digit 5'
1107,Mnist_1108,A vast desert with a towering canyon in the di...,low quality,1.5,0,No illusion,No illusion,No illusion,No illusion,'digit 0'


In [7]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [8]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'3',
 'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [9]:
{item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}

{'no illusion'}

In [10]:
{item.strip().lower() for item in classes[dataset]}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [11]:
if dataset == 'Mnist':
    sum(df[f'LLaVa_Mnist_raw_images']=='3')
    df[f'LLaVa_Mnist_raw_images'].unique()

In [12]:
if dataset == 'Mnist':
    df.loc[df['LLaVa_Mnist_raw_images'] == ' 3', 'LLaVa_Mnist_raw_images'] = 'digit 3'

In [13]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

## Raw

In [14]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'LLaVa_{dataset}_raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [15]:
idx2label

{0: 'digit 0',
 1: 'digit 1',
 2: 'digit 2',
 3: 'digit 3',
 4: 'digit 4',
 5: 'digit 5',
 6: 'digit 6',
 7: 'digit 7',
 8: 'digit 8',
 9: 'digit 9',
 10: 'no illusion'}

In [16]:
set(list(y_pred))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [17]:
set(list(y_true))

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9'}

In [18]:
unique_classes = [item.strip().replace('\'', '').lower() for item in classes[dataset]]
for item in y_pred:
    assert item in unique_classes

In [19]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.5545536519386834
Precision: 0.8351511568646648
Recall: 0.5666176610950538
F1 score: 0.597540674940667


In [20]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 55.46
Precision: 83.52
Recall: 56.66
F1 score: 59.75


## Illusion

In [21]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

y_pred = list(df[f'LLaVa_{dataset}_ill_images'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images'].apply(lambda x: x.strip().lower()))

In [22]:
set(y_pred)

{'no illusion'}

In [23]:

set(y_true)

{'digit 0',
 'digit 1',
 'digit 2',
 'digit 3',
 'digit 4',
 'digit 5',
 'digit 6',
 'digit 7',
 'digit 8',
 'digit 9',
 'no illusion'}

In [24]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.09023789991796555
Precision: 0.008203445447087777
Recall: 0.09090909090909091
F1 score: 0.015048908954100828


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 9.02
Precision: 0.82
Recall: 9.09
F1 score: 1.5


## Illusion Filtered

In [26]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'LLaVa_{dataset}_illusion_images_filtered'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images_filtered'].apply(lambda x: x.strip().lower()))

In [27]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.09023789991796555
Precision: 0.008203445447087777
Recall: 0.09090909090909091
F1 score: 0.015048908954100828


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 9.02
Precision: 0.82
Recall: 9.09
F1 score: 1.5


# FashionMnist

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'FashionMnist'

df = pd.read_csv(f'LLaVa_{dataset}_results.csv')

In [30]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [31]:
illusionless_indexes

[66,
 878,
 988,
 30,
 422,
 947,
 1006,
 568,
 328,
 70,
 1066,
 1003,
 671,
 155,
 511,
 739,
 91,
 860,
 284,
 727,
 781,
 862,
 580,
 537,
 935,
 357,
 620,
 742,
 272,
 490,
 900,
 768,
 90,
 8,
 482,
 274,
 399,
 1098,
 749,
 491,
 643,
 1124,
 922,
 892,
 962,
 133,
 664,
 1027,
 320,
 459,
 845,
 488,
 75,
 65,
 1017,
 616,
 147,
 1092,
 165,
 306,
 787,
 766,
 231,
 197,
 904,
 340,
 391,
 714,
 887,
 848,
 912,
 502,
 562,
 293,
 1068,
 365,
 244,
 546,
 933,
 619,
 337,
 356,
 360,
 983,
 711,
 890,
 458,
 10,
 1115,
 89,
 679,
 654,
 496,
 163,
 536,
 917,
 829,
 324,
 799,
 1015,
 494,
 1075,
 556,
 1065,
 1023,
 128,
 1002,
 945,
 821,
 278,
 861,
 1108,
 715,
 1103,
 794]

In [32]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,LLaVa_FashionMnist_ill_images,LLaVa_FashionMnist_illusion_images_filtered,LLaVa_FashionMnist_illusionless_images,LLaVa_FashionMnist_illusionless_images_filtered,LLaVa_FashionMnist_raw_images
0,FashionMnist_1,Two friends hiking on a mountain during sunset,low quality,1.5,9,No illusion,No illusion,No illusion,No illusion,Sneaker
1,FashionMnist_2,A starry night sky with constellations glowing,low quality,1.5,2,No illusion,No illusion,No illusion,No illusion,T-shirt/top
2,FashionMnist_3,A Thanksgiving feast giving thanks and love,low quality,1.5,1,No illusion,No illusion,No illusion,No illusion,Trouser
3,FashionMnist_4,A gecko on a large leaf in a tropical rainforest,low quality,1.5,1,No illusion,No illusion,No illusion,No illusion,Trouser
4,FashionMnist_5,A Hanukkah menorah lighting symbolizing faith,low quality,1.5,6,No illusion,T-shirt/top,No illusion,No illusion,T-shirt/top
...,...,...,...,...,...,...,...,...,...,...
1147,FashionMnist_1148,"Electrified waterfalls cascade, neon rivers re...",low quality,1.5,2,No illusion,T-shirt/top,No illusion,No illusion,T-shirt/top
1148,FashionMnist_1149,A rocky desert landscape with cacti and tumble...,low quality,1.5,2,No illusion,No illusion,No illusion,No illusion,T-shirt/top
1149,FashionMnist_1150,A majestic waterfall plunging into a crystal-c...,low quality,1.5,5,No illusion,No illusion,No illusion,No illusion,Sneaker
1150,FashionMnist_1151,"Eerie silence surrounds snowy mountains, glowi...",low quality,1.5,3,No illusion,No illusion,No illusion,No illusion,T-shirt/top


In [33]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [34]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'sandal',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [35]:
{item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}

{'dress', 'no illusion', 't-shirt/top'}

In [36]:
{item.strip().lower() for item in classes[dataset]}

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'no illusion',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [37]:
if dataset == 'Mnist':
    sum(df[f'LLaVa_Mnist_raw_images']=='3')
    df[f'LLaVa_Mnist_raw_images'].unique()

In [38]:
if dataset == 'Mnist':
    df.loc[df['LLaVa_Mnist_raw_images'] == ' 3', 'LLaVa_Mnist_raw_images'] = 'digit 3'

In [39]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'sandal',
 'sneaker',
 't-shirt/top',
 'trouser'}

## Raw

In [40]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'LLaVa_{dataset}_raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [41]:
idx2label

{0: 't-shirt/top',
 1: 'trouser',
 2: 'pullover',
 3: 'dress',
 4: 'coat',
 5: 'sandal',
 6: 'shirt',
 7: 'sneaker',
 8: 'bag',
 9: 'ankle boot',
 10: 'no illusion'}

In [42]:
set(list(y_pred))

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'sandal',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [43]:
set(list(y_true))

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [44]:
unique_classes = [item.strip().replace('\'', '').lower() for item in classes[dataset]]
for item in y_pred:
    assert item in unique_classes

In [45]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.3862847222222222
Precision: 0.6056217278482896
Recall: 0.3933578591601197
F1 score: 0.3837707315612987


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 38.63
Precision: 60.56
Recall: 39.34
F1 score: 38.38


## Illusion

In [47]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

y_pred = list(df[f'LLaVa_{dataset}_ill_images'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images'].apply(lambda x: x.strip().lower()))

In [48]:
set(y_pred)

{'dress', 'no illusion', 't-shirt/top'}

In [49]:

set(y_true)

{'ankle boot',
 'bag',
 'coat',
 'dress',
 'no illusion',
 'pullover',
 'sandal',
 'shirt',
 'sneaker',
 't-shirt/top',
 'trouser'}

In [50]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.08997632202052092
Precision: 0.008212073188301397
Recall: 0.09011857707509882
F1 score: 0.015052485640720932


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 9.0
Precision: 0.82
Recall: 9.01
F1 score: 1.51


## Illusion Filtered

In [52]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'LLaVa_{dataset}_illusion_images_filtered'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images_filtered'].apply(lambda x: x.strip().lower()))

In [53]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.13733228097868982
Precision: 0.26900091475984333
Recall: 0.13722926750449685
F1 score: 0.07547476429597136


In [54]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 13.73
Precision: 26.9
Recall: 13.72
F1 score: 7.55


# IllusionAnimals

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'IllusionAnimals'

df = pd.read_csv(f'LLaVa_{dataset}_results.csv')

In [56]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [57]:
illusionless_indexes

[585,
 33,
 439,
 494,
 591,
 15,
 211,
 473,
 832,
 503,
 843,
 284,
 669,
 830,
 164,
 35,
 533,
 501,
 335,
 77,
 255,
 975,
 978,
 763,
 369,
 45,
 430,
 880,
 142,
 617,
 363,
 390,
 431,
 290,
 846,
 691,
 268,
 467,
 178,
 702,
 310,
 677,
 371,
 136,
 962,
 786,
 972,
 245,
 450,
 628,
 384,
 974,
 596,
 4,
 241,
 137,
 199,
 931,
 959,
 549,
 374,
 790,
 919,
 952,
 321,
 682,
 562,
 461,
 446,
 481,
 66,
 668,
 598,
 332,
 864,
 513,
 160,
 862,
 229,
 422,
 244,
 37,
 32,
 508,
 308,
 837,
 621,
 673,
 73,
 546,
 875,
 82,
 153,
 393,
 580,
 383,
 615,
 907,
 115,
 794]

In [58]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,LLaVa_IllusionAnimals_ill_images,LLaVa_IllusionAnimals_illusion_images_filtered,LLaVa_IllusionAnimals_illusionless_images,LLaVa_IllusionAnimals_illusionless_images_filtered,LLaVa_IllusionAnimals_raw_images
0,IllusionAnimals_1,A raging river flowing through a dense jungle ...,low quality,2.5,cat,No illusion,Cat,No illusion,No illusion,Cat
1,IllusionAnimals_2,A starry night sky over a tranquil lake,low quality,2.5,cat,No illusion,No illusion,No illusion,No illusion,Cat
2,IllusionAnimals_3,Jaguar (Panthera Onca) patrolling South Americ...,low quality,2.5,cat,No illusion,No illusion,No illusion,No illusion,Cat
3,IllusionAnimals_4,"Flowing lava illuminates cavern walls, ancient...",low quality,2.5,cat,No illusion,No illusion,No illusion,No illusion,Cat
4,IllusionAnimals_5,"Dense forest canopy, sunlight filters through,...",low quality,2.5,cat,No illusion,Cat,No illusion,No illusion,Cat
...,...,...,...,...,...,...,...,...,...,...
995,IllusionAnimals_996,Saharan sandstorm turning daytime into darkness,low quality,2.5,rooster,Rooster,Rooster,No illusion,No illusion,Rooster
996,IllusionAnimals_997,Urban city with skyscrapers and traffic,low quality,2.5,rooster,No illusion,Rooster,No illusion,No illusion,Rooster
997,IllusionAnimals_998,A serene meadow with wildflowers and butterflies,low quality,2.5,rooster,No illusion,Rooster,No illusion,No illusion,Rooster
998,IllusionAnimals_999,Elk bugling in crisp autumn air,low quality,2.5,rooster,Deer,Rooster,Deer,Deer,Rooster


In [59]:
raw_classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' ],
}

classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'No illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'No illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster', 'No illusion' ],
}

# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_ill_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusion_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images_filtered'].unique()}.issubset({item.strip().lower() for item in classes[dataset]})
# assert {item.strip().lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}.issubset({item.strip().lower() for item in raw_classes[dataset]})

In [60]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [61]:
{item.strip().lower() for item in df[f'LLaVa_{dataset}_illusionless_images'].unique()}

{'butterfly',
 'deer',
 'dog',
 'dolphin',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'snake',
 'swan'}

In [62]:
{item.strip().lower() for item in classes[dataset]}

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [63]:
if dataset == 'Mnist':
    sum(df[f'LLaVa_Mnist_raw_images']=='3')
    df[f'LLaVa_Mnist_raw_images'].unique()

In [64]:
if dataset == 'Mnist':
    df.loc[df['LLaVa_Mnist_raw_images'] == ' 3', 'LLaVa_Mnist_raw_images'] = 'digit 3'

In [65]:
{item.strip().replace('\'', '').lower() for item in df[f'LLaVa_{dataset}_raw_images'].unique()}

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

## Raw

In [66]:
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = df['label'].apply(lambda x: x.strip().lower())
else:
    y_true = df['label'].apply(lambda x: idx2label[x].strip().lower())
y_pred = df[f'LLaVa_{dataset}_raw_images'].apply(lambda x: x.strip().replace('\'', '').lower())

In [67]:
idx2label

{0: 'cat',
 1: 'dog',
 2: 'pigeon',
 3: 'butterfly',
 4: 'elephant',
 5: 'horse',
 6: 'deer',
 7: 'snake',
 8: 'fish',
 9: 'rooster',
 10: 'no illusion'}

In [68]:
set(list(y_pred))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [69]:
set(list(y_true))

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'pigeon',
 'rooster',
 'snake'}

In [70]:
unique_classes = [item.strip().replace('\'', '').lower() for item in classes[dataset]]
for item in y_pred:
    assert item in unique_classes

In [71]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0


In [72]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 100.0
Precision: 100.0
Recall: 100.0
F1 score: 100.0


## Illusion

In [73]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))

# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))

y_pred = list(df[f'LLaVa_{dataset}_ill_images'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images'].apply(lambda x: x.strip().lower()))

In [74]:
set(y_pred)

{'butterfly',
 'deer',
 'dog',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [75]:

set(y_true)

{'butterfly',
 'cat',
 'deer',
 'dog',
 'elephant',
 'fish',
 'horse',
 'no illusion',
 'pigeon',
 'rooster',
 'snake'}

In [76]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.1390909090909091
Precision: 0.4561371177867544
Recall: 0.1390909090909091
F1 score: 0.1042425761570259


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 13.91
Precision: 45.61
Recall: 13.91
F1 score: 10.42


## Illusion Filtered

In [78]:
classes = {
    'FashionMnist': ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot', 'no illusion'],
    'Mnist': ['digit 0', 'digit 1', 'digit 2', 'digit 3', 'digit 4', 'digit 5', 'digit 6', 'digit 7', 'digit 8', 'digit 9', 'no illusion'],
    'IllusionAnimals': ['cat', 'dog', 'pigeon', 'butterfly', 'elephant', 'horse', 'deer', 'snake', 'fish', 'rooster' , 'no illusion'],
}
idx2label = {idx: label.strip().lower() for idx, label in enumerate(classes[dataset])}
# y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df['label'].apply(lambda x: 'no illusion'))
if dataset == 'IllusionAnimals':
    y_true = list(df['label'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))
else:
    y_true = list(df['label'].apply(lambda x: idx2label[x].strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)['label'].apply(lambda x: 'no illusion'))


y_pred = list(df[f'LLaVa_{dataset}_illusion_images_filtered'].apply(lambda x: x.strip().lower())) + list(df.filter(items=illusionless_indexes, axis=0)[f'LLaVa_{dataset}_illusionless_images_filtered'].apply(lambda x: x.strip().lower()))

In [79]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.5118181818181818
Precision: 0.9009059037649784
Recall: 0.5118181818181818
F1 score: 0.5282640082042331


In [80]:
print("Accuracy:", round(accuracy*100,2))
print("Precision:", round(precision*100,2))
print("Recall:", round(recall*100,2))
print("F1 score:", round(f1*100,2))

Accuracy: 51.18
Precision: 90.09
Recall: 51.18
F1 score: 52.83
